# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [17]:
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.automl import AutoMLConfig
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from trainCovid19Dataset import clean_data
import os
import shutil

## Initialize Workspace

Create a workspace, if it doesn't exist, using the AzureML SDK

In [13]:
ws = Workspace.from_config()
    
ws.get_details()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-139794
Azure region: southcentralus
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-139794


## Cluster

Get cluster if it exists else create one

In [14]:
amlcompute_cluster_name = "Covid19Cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

In [15]:
ws = Workspace.from_config()
experiment_name = 'Covid19HyperdriveEx'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [19]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.2, evaluation_interval=2, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( 
    {
        '--C': choice(0.01, 0.1, 1, 10, 100), 
        '--max_iter': choice(25, 50, 100,150)
    }
)
if "training" not in os.listdir():
    os.mkdir("./training")
script_folder = './training/'    
os.makedirs(script_folder, exist_ok=True)
shutil.copy('trainCovid19Dataset.py', script_folder)

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory=script_folder, compute_target= amlcompute_cluster_name, entry_script='trainCovid19Dataset.py')

hyperdrive_run_config = HyperDriveConfig(
                                   hyperparameter_sampling = param_sampling,
                                   primary_metric_name = 'Accuracy',
                                   primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                   max_total_runs=30,
                                   max_concurrent_runs = 3,
                                   policy = early_termination_policy,
                                   estimator = estimator)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [ ]:
#TODO: Submit your experiment

In [ ]:
# Specify parameter sampler
ps = RandomParameterSampling( 
    {
        '--C': choice(0.01, 0.1, 1, 10, 100), 
        '--max_iter': choice(25, 50, 100,150)
    }
)

# Specify a Policy
#any training runs whose best metric at interval 5 is less than 0.66 (1/(1+slack_factor)) will be terminated.
policy = BanditPolicy(slack_factor = 0.2, evaluation_interval=2, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")
script_folder = './training/'    
os.makedirs(script_folder, exist_ok=True)
shutil.copy('train.py', script_folder)

# Create a SKLearn estimator for use with train.py
# ************ 'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.***********
est = SKLearn(source_directory=script_folder, compute_target= cpu_cluster, entry_script='train.py')
# est = ScriptRunConfig(
#     source_directory='.',
#     script='train.py',
#     compute_target=cpu_cluster,
#     )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                                   hyperparameter_sampling = ps,
                                   primary_metric_name = 'Accuracy',
                                   primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                   max_total_runs=30,
                                   max_concurrent_runs = 3,
                                   policy = policy,
                                   estimator = est)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [20]:
import joblib
# Get your best run and save the model from that run.
best_run_sklearn = hyperdrive_run.get_best_run_by_primary_metric()
# joblib.dump(best_run_sklearn, 'training/sklearn_bankmarketing_model.joblib')
SKLearn_Model = best_run_sklearn.register_model(model_name="sklearnBankmarketingModel", model_path='outputs/model.joblib')
# Get the metrics of the bestselected run
best_run_metrics = best_run_sklearn.get_metrics()
# Show the Accuracy of that run
print('Best Accuracy: {}'.format(best_run_metrics['Accuracy']))
best_run_sklearn

NameError: name 'hyperdrive_run' is not defined

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service